In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from src.model import *
# from src.util import *
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/kaggle_miscrosoft/'
best_trained_model = '2022-01-18 19:10/50w_epoch:15_test_acc:0.961822.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/kaggle/"
train_data_path = data_path + "malwares/"  # Training data
train_label_path = data_path + "train_labels.csv"  # Training label

In [3]:
result_path = '/workdir/security/home/junjiehuang2468/paper/results/kaggle_miscrosoft/'

In [4]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 16  # Number of cores to use for data loader
BATCH_SIZE = 3  #
LEAVE_BIT_NUMBER = 500000
KERNEL_SIZE = 500  # Kernel size & stride for Malconv (defualt : 500)

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
def mp_func(i,inpu,te,gr):
    check = 0
    grad_cum = 0
    for j,(inp,g,t) in enumerate(zip(inpu,gr,te)):
        if inp != 0: 
            check = j
            continue
        max_idx = np.argmin(g).tolist()
        org_max_idx = np.argmax(t).tolist()
        if g[max_idx] > 0: continue
        grad_cum += g[max_idx]
        te[j][org_max_idx] = 0
        te[j][max_idx] = 1
    return [i,te,check,grad_cum]

In [10]:
# class Model(nn.Module):
#     def __init__(self, data_length = 2e6, kernel_size = 500):
#         super().__init__()
#         self.embedding = nn.Embedding(257, 8, padding_idx=0)
#         self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         # self.bn_1 = nn.BatchNorm1d(128)
#         self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
#         self.fc_layer_3 = nn.Linear(128, 128)
#         self.fc_layer_4 = nn.Linear(128, 9)
        
#     def forward(self,x):
#         x = self.embedding(x)
#         x = x.transpose(-1,-2)
#         x_conv_1 = self.conv_layer_1(x[:,:4,:])
#         x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
#         x = x_conv_1*x_conv_2
#         del x_conv_1,x_conv_2
#         x = self.pool_layer_2(x).squeeze()
#         x = self.fc_layer_3(x)
#         x = self.fc_layer_4(x)
#         # x = torch.sigmoid(x)
#         return x

In [11]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 9)
    
    def forward(self, input_, loss_fn, fake_label, label):
        temp = F.one_hot(input_,num_classes=257).float()
        temp.requires_grad = True
        temp.retain_grad()
        for _ in range(10):
            x = temp @ self.embedding.weight
            x = x.transpose(-1,-2)
            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))
            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            # fake_label = torch.zeros_like(x)
            print((torch.argmax(torch.softmax(x,dim=-1),dim=-1).tolist() , label.tolist()))
            loss = loss_fn(x,fake_label).cuda()
            print(loss)
            loss.backward()
            data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
                input_.detach().cpu().numpy(),
                temp.detach().cpu().numpy(),
                temp.grad.detach().cpu().numpy()
            ))]
            with mp.Pool(processes=24 if len(data) > 24 else len(data)) as pool:
                results = pool.starmap(mp_func,data)
            
            check = [r[2] for r in results]
            results = sorted(results,key = lambda x: x[0])
            print([(r[2],temp.grad[i,r[2]:,:].sum(-1).sum(-1).tolist()) for i,r in enumerate(results)])
            if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: break
            for i in range(len(temp)):
                temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
                
        return x.cpu().detach().numpy(),temp,check

In [12]:
# class Model(nn.Module):
#     def __init__(self, data_length = 2e6, kernel_size = 500):
#         super().__init__()
#         self.embedding = nn.Embedding(257, 8, padding_idx=0)
#         self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         # self.bn_1 = nn.BatchNorm1d(128)
#         self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
#         self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
#         self.fc_layer_3 = nn.Linear(128, 128)
#         self.fc_layer_4 = nn.Linear(128, 9)
    
#     def forward(self, input_, loss_fn):
#         temp = F.one_hot(input_,num_classes=257).float()
#         temp.requires_grad = True
#         temp.retain_grad()
#         for _ in range(6):
#             x = temp @ self.embedding.weight
#             x = torch.transpose(x, -1, -2)

#             x_conv_1 = self.conv_layer_1(x[:,:4,:])
#             x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

#             x = x_conv_1*x_conv_2
#             del x_conv_1,x_conv_2
#             x = self.pool_layer_2(x).squeeze()
            
#             x = self.fc_layer_3(x)
#             x = self.fc_layer_4(x)
            
#             print(torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean())
            
#             mislead_labels = torch.zeros_like(x).cuda()
#             loss = loss_fn(x,mislead_labels)
#             loss.backward()
            
#             data = [(i,inpu,te,gr) for i,(inpu,te,gr) in enumerate(zip(
#                 input_.detach().cpu().numpy(),
#                 temp.detach().cpu().numpy(),
#                 temp.grad.detach().cpu().numpy()
#             ))]
#             with mp.Pool(processes=24 if len(data) > 24 else len(data)) as pool:
#                 results = pool.starmap(mp_func,data)
#             results = sorted(results,key = lambda x: x[0])
#             for i in range(len(temp)):
#                 temp.data[i] = torch.tensor(results[i][1], dtype=torch.float, requires_grad=True).cuda()
        
#         return x.detach().cpu().numpy(),temp

In [13]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [14]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [15]:
if __name__=='__main__':
    model.eval()
    acc = []
    preds = []
    labels = []
    for step, (batch_data,batch_label) in enumerate(tqdm(validloader)):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze() - 1
        fake_label = torch.zeros_like(batch_label)
        pred,temp,check = model(batch_data,ce_loss,fake_label,batch_label)
        if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: continue
            
        pred = np.argmax(pred,1)
        batch_label = batch_label.cpu().data.numpy()

        preds.extend(pred.tolist())
        labels.extend(batch_label.tolist())

        count = total = 0
        for c,ans in zip(check,batch_label == pred):
            if c != LEAVE_BIT_NUMBER-1:
                count += ans
                total += 1
        acc.append(count/total)
        print(f"test：{count/total}, test mean: {np.mean(acc)}")

  0%|          | 0/725 [00:00<?, ?it/s]

([6, 0, 2], [6, 7, 2])
tensor(7.9986, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 1/725 [00:31<6:25:11, 31.92s/it]

([8, 2, 8], [8, 2, 8])
tensor(17.3843, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 2/725 [00:57<5:38:11, 28.07s/it]

...

test：1.0, test mean: 0.3387096774193548
([8, 2, 2], [8, 2, 2])
tensor(19.7729, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -11.809257507324219), (499999, 0.0), (499999, 0.0)]
([8, 2, 2], [8, 2, 2])
tensor(20.0988, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -20.774173736572266), (499999, 0.0), (499999, 0.0)]
([8, 2, 2], [8, 2, 2])
tensor(20.0154, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -34.729347229003906), (499999, 0.0), (499999, 0.0)]
([8, 2, 2], [8, 2, 2])
tensor(19.3392, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -45.35823059082031), (499999, 0.0), (499999, 0.0)]
([0, 2, 2], [8, 2, 2])
tensor(19.0394, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -51.3652458190918), (499999, 0.0), (499999, 0.0)]
([0, 2, 2], [8, 2, 2])
tensor(18.9360, device='cuda:0', grad_fn=<NllLossBackward0>)
[(454655, -54.071434020996094), (499999, 0.0), (499999, 0.0)]
([0, 2, 2], [8, 2, 2])
tensor(18.9132, device='cuda:0', grad_fn=<NllLossBackward0>)
[(4

 11%|█         | 79/725 [2:42:57<31:29:41, 175.51s/it]

test：0.0, test mean: 0.328125
([2, 0, 1], [2, 0, 1])
tensor(15.5106, device='cuda:0', grad_fn=<NllLossBackward0>)


Process ForkPoolWorker-1119:
Process ForkPoolWorker-1118:
Process ForkPoolWorker-1120:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
 11%|█         | 79/725 [2:43:17<22:15:18, 124.02s/it]


KeyboardInterrupt: 

In [14]:
if __name__=='__main__':
    model.eval()
    acc = []
    preds = []
    labels = []
    for step, (batch_data,batch_label) in enumerate(tqdm(validloader)):
        batch_data = batch_data.cuda() if CUDA else batch_data
        batch_label = batch_label.cuda() if CUDA else batch_label
        batch_label = batch_label.squeeze() - 1
        fake_label = torch.zeros_like(batch_label)
        pred,temp,check = model(batch_data,ce_loss,fake_label,batch_label)
        if sum(check)/len(check) == LEAVE_BIT_NUMBER-1: continue
            
        pred = np.argmax(pred,1)
        batch_label = batch_label.cpu().data.numpy()

        preds.extend(pred.tolist())
        labels.extend(batch_label.tolist())

        count = total = 0
        for c,ans in zip(check,batch_label == pred):
            if c != LEAVE_BIT_NUMBER-1:
                count += ans
                total += 1
        acc.append(count/total)
        print(f"test：{count/total}, test mean: {np.mean(acc)}")

  0%|          | 0/725 [00:00<?, ?it/s]

([1, 0, 5], [1, 0, 5])
tensor(10.2925, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 1/725 [00:27<5:30:51, 27.42s/it]

([1, 0, 2], [1, 0, 2])
tensor(14.2205, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, 0.0), (499999, 0.0), (499999, 0.0)]


  0%|          | 2/725 [00:52<5:13:44, 26.04s/it]

...

100%|█████████▉| 723/725 [23:42:02<05:58, 179.40s/it]

test：0.0, test mean: 0.2952755905511811
([7, 2, 0], [7, 2, 0])
tensor(16.2474, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 2.7659714221954346), (499999, 0.0), (442367, 4.592854565999005e-06)]
([7, 2, 0], [7, 2, 0])
tensor(13.8536, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 20.05219268798828), (499999, 0.0), (442367, 0.00017068447778001428)]
([7, 2, 0], [7, 2, 0])
tensor(10.0640, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 48.10798645019531), (499999, 0.0), (442367, 0.00016925446107052267)]
([0, 2, 0], [7, 2, 0])
tensor(9.7079, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 48.1080322265625), (499999, 0.0), (442367, 0.00016924337251111865)]
([0, 2, 0], [7, 2, 0])
tensor(9.7079, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 48.10807800292969), (499999, 0.0), (442367, 0.00016923289513215423)]
([0, 2, 0], [7, 2, 0])
tensor(9.7079, device='cuda:0', grad_fn=<NllLossBackward0>)
[(368639, 48.108123779296875), (499999, 0.0), (442367, 0.00016926

100%|█████████▉| 724/725 [23:46:55<03:33, 213.75s/it]

test：0.5, test mean: 0.296078431372549
([0, 2], [0, 2])
tensor(13.4703, device='cuda:0', grad_fn=<NllLossBackward0>)
[(499999, -1.418677857145667e-05), (499999, 0.0)]


100%|██████████| 725/725 [23:47:17<00:00, 118.12s/it]


In [15]:
# if __name__=='__main__':
#     acc = []
#     preds = []
#     labels = []

#     for step, val_batch_data in enumerate(validloader):
#         print(step,len(validloader))
#         cur_batch_size = val_batch_data[0].size(0)

#         exe_input = val_batch_data[0].to(device) if use_gpu else val_batch_data[0]

#         label = val_batch_data[1].to(device) if use_gpu else val_batch_data[1]
#         fake_label = torch.zeros_like(label.squeeze())
#         # fake_label[fake_label == 9] = 0
#         # temp = torch.zeros((6,9))
#         # for i,j in enumerate(fake_label): temp[i][j] = 1
#         # fake_label = temp.cuda()
#         # fake_label = fake_label.reshape(-1,1)
#         label = label.squeeze() - 1
        
#         pred,temp = malconv(exe_input,ce_loss,fake_label,label)
        
#         pred = np.argmax(pred,1)
#         label = label.cpu().data.numpy().astype(int)
#         # temp_grad,temp = malconv(exe_input)
#         preds.extend(pred.tolist())
#         labels.extend(label.tolist())

#         val_Macc = (label == pred).sum()
#         val_Macc = val_Macc / cur_batch_size
#         acc.append(val_Macc)
#         print(f"test：{val_Macc}, test mean: {np.mean(acc)}")